Load walk+transit network

In [1]:
import pandana as pdn

pt_network = pdn.Network.from_hdf5('data/transit_24h/transit_0820.h5')

In [2]:
O_lat,O_lon=[43.30141078225517, -1.973559641657051]

O_nodes = pt_network.get_node_ids([O_lon], [O_lat]).values
O_nodes

#D_lat,D_lon=[43.29233260460531, -1.9688443479414162] #hospital
#D_lat,D_lon=[43.31362186877746, -2.005299988731995] #ondarreta
#D_lat,D_lon=[43.30547505704917, -2.0167963423075483] #berio 
D_lat,D_lon=[43.302228882526784, -2.2050072959980493] #balenciaga

D_nodes = pt_network.get_node_ids([D_lon], [D_lat]).values
D_nodes

# Convert both node arrays to integers (if they are not already)
O_nodes = O_nodes.astype(int)
D_nodes = D_nodes.astype(int)

# Now calculate the shortest path length
pt_network.shortest_path_length(O_nodes[0], D_nodes[0])

35.126

In [3]:
import geopandas as gpd
import json
from shapely.geometry import LineString

# Define origin and destination coordinates
O_lat, O_lon = [43.30450961354353, -2.0157260103962353]
D_lat, D_lon = [43.29235378940283, -1.9687827045051787]

# Get nearest nodes
O_nodes = pt_network.get_node_ids([O_lon], [O_lat]).values.astype(int)
D_nodes = pt_network.get_node_ids([D_lon], [D_lat]).values.astype(int)

# Calculate the shortest path
route_nodes = pt_network.shortest_path(O_nodes[0], D_nodes[0])

# Try to get node coordinates
try:
    # If pt_network stores nodes as a dictionary
    route_coords = [(pt_network.nodes[node][0], pt_network.nodes[node][1]) for node in route_nodes]
except AttributeError:
    # If pt_network has a nodes DataFrame
    route_coords = [(pt_network.nodes_df.loc[node, "x"], pt_network.nodes_df.loc[node, "y"]) for node in route_nodes]

# Convert the route to a LineString geometry
route_geometry = LineString(route_coords)

# Create a GeoDataFrame
route_gdf = gpd.GeoDataFrame([{"geometry": route_geometry}], crs="EPSG:4326")

# Save to GeoJSON
route_gdf.to_file("route.geojson", driver="GeoJSON")

print("Route saved to route.geojson")

Route saved to route.geojson


Load nodes gdf

In [2]:
import geopandas as gpd

# Load the geojson file and print to check
nodes_gdf = gpd.read_file('./data/geojson/building_nodes_with_poi.geojson')

print(nodes_gdf.head())

  Referencia       lon        lat  nearest_node  time_to_nearest_poi  \
0    8594095 -1.948183  43.299723    2135506826             5.202316   
1    8594099 -1.948484  43.299120    2135506826             5.202316   
2    8594100 -1.948365  43.298748    2135506826             5.202316   
3    8796114 -1.928046  43.315046    6548202929             3.369070   
4    8796136 -1.928175  43.313904    6548202929             2.023299   

  block_color nearest_poi  nearest_poi_node                   geometry  
0      yellow     20-0004         457239233  POINT (-1.94818 43.29972)  
1      yellow     20-0004         457239233  POINT (-1.94848 43.29912)  
2      yellow     20-0004         457239233  POINT (-1.94837 43.29875)  
3       green     20-0178         533746770  POINT (-1.92805 43.31505)  
4       green     20-0019         476567754   POINT (-1.92818 43.3139)  


In [5]:
print(len(nodes_gdf)) # should be 58768

58768


In [6]:
# Fill the time_to_poi column with None
nodes_gdf['nearest_node'] = None
nodes_gdf['time_to_nearest_poi'] = None
nodes_gdf['block_color'] = None
nodes_gdf['nearest_poi'] = None
nodes_gdf['nearest_poi_node'] = None
print(nodes_gdf.head())

  Referencia       lon        lat nearest_node time_to_nearest_poi  \
0    8594095 -1.948183  43.299723         None                None   
1    8594099 -1.948484  43.299120         None                None   
2    8594100 -1.948365  43.298748         None                None   
3    8796114 -1.928046  43.315046         None                None   
4    8796136 -1.928175  43.313904         None                None   

  block_color nearest_poi nearest_poi_node                   geometry  
0        None        None             None  POINT (-1.94818 43.29972)  
1        None        None             None  POINT (-1.94848 43.29912)  
2        None        None             None  POINT (-1.94837 43.29875)  
3        None        None             None  POINT (-1.92805 43.31505)  
4        None        None             None   POINT (-1.92818 43.3139)  


Load POI gdf

In [1]:
poi = gpd.read_file('./data/poi/estetica.geojson')
print(poi.head())

NameError: name 'gpd' is not defined

In [8]:
poi['lon'] = poi['geometry'].x
poi['lat'] = poi['geometry'].y

Create functions to find nearest node

In [9]:
import pandas as pd

def assign_nearest_nodes(gdf, pt_network):
    # Ensure lon/lat are numeric
    gdf['lon'] = pd.to_numeric(gdf['lon'], errors='coerce')
    gdf['lat'] = pd.to_numeric(gdf['lat'], errors='coerce')

    # Drop rows with missing coordinates
    gdf = gdf.dropna(subset=['lon', 'lat']).copy()
    
    # Convert to numpy arrays
    lon_values = gdf['lon'].to_numpy()
    lat_values = gdf['lat'].to_numpy()

    # Assign nearest nodes
    try:
        gdf['nearest_node'] = pt_network.get_node_ids(lon_values, lat_values)
    except Exception as e:
        print(f"Error assigning nearest nodes: {e}")
        gdf['nearest_node'] = None  # Assign None in case of failure

    return gdf

In [10]:
nodes_gdf = assign_nearest_nodes(nodes_gdf, pt_network)
print(nodes_gdf.head())

  Referencia       lon        lat  nearest_node time_to_nearest_poi  \
0    8594095 -1.948183  43.299723         29675                None   
1    8594099 -1.948484  43.299120         29675                None   
2    8594100 -1.948365  43.298748         29675                None   
3    8796114 -1.928046  43.315046         64984                None   
4    8796136 -1.928175  43.313904         64984                None   

  block_color nearest_poi nearest_poi_node                   geometry  
0        None        None             None  POINT (-1.94818 43.29972)  
1        None        None             None  POINT (-1.94848 43.29912)  
2        None        None             None  POINT (-1.94837 43.29875)  
3        None        None             None  POINT (-1.92805 43.31505)  
4        None        None             None   POINT (-1.92818 43.3139)  


In [11]:
poi = assign_nearest_nodes(poi, pt_network)
print(poi.head())

  IDENTIFICADOR     DENOMINACIÓN DEL ESTABLECIMIENTO              DIRECCIÓN  \
0      L0328934  Kix - Kax Ileapaindegi eta Estetika  Calle Armendi Gain, 1   
1      L0432423                           Peluquería     Calle San Juan, 39   
2      L0310711                Peluquería y estética     Calle San Juan, 30   
3      L0190385               Larraitz Ileapaindegia   Barrio Larraitz, 2-A   
4      L0417371                Kixkur ile-apaindegia    Barrio Larraitz, 14   

    LOCALIDAD  CÓD. POSTAL MUNICIPIO TERRITORIO HISTÓRICO  CNAE  \
0         Aia        20809       Aia             Gipuzkoa  9602   
1      Alegia        20260    Alegia             Gipuzkoa  9602   
2      Alegia        20260    Alegia             Gipuzkoa  9602   
3  Errotaldea        20260    Alegia             Gipuzkoa  9602   
4  Errotaldea        20260    Alegia             Gipuzkoa  9602   

                             DESCRIPCIÓN CNAE ESTRATO EMPLEO  ...        lat  \
0  Peluquería y otros tratamientos de bell

Find nearest POI for each node

In [12]:
from shapely.geometry import Point

def nearest_poi(row, poi_df):
    # Asegúrate de usar la geometría para calcular la distancia
    distances = poi['geometry'].apply(lambda x: row.geometry.distance(x))
    nearest = poi.loc[distances.idxmin()]
    return nearest['DENOMINACIÓN DEL ESTABLECIMIENTO']  # O el identificador que quieras retornar

# Aplica la función sobre cada fila del GeoDataFrame
nodes_gdf['nearest_poi'] = nodes_gdf.apply(nearest_poi, poi_df=poi, axis=1)

KeyboardInterrupt: 

Assign the nearest node of the nearest POI

In [ ]:
nodes_gdf['nearest_poi_node'] = None

# Get the nearest node of the nearest poi that is already in the poi dataframe
def nearest_poi_node(row):
    nearest_node = poi.loc[row['nearest_poi'] == poi['DENOMINACIÓN DEL ESTABLECIMIENTO']]['nearest_node']
    return nearest_node.values[0]

In [ ]:
# Apply the function to each row in the GeoDataFrame
nodes_gdf['nearest_poi_node'] = nodes_gdf.apply(nearest_poi_node, axis=1)

In [ ]:
print(nodes_gdf.head())

  Referencia       lon        lat  nearest_node time_to_nearest_poi  \
0    8594095 -1.948183  43.299723         29675                None   
1    8594099 -1.948484  43.299120         29675                None   
2    8594100 -1.948365  43.298748         29675                None   
3    8796114 -1.928046  43.315046         64984                None   
4    8796136 -1.928175  43.313904         64984                None   

  block_color                 nearest_poi  nearest_poi_node  \
0        None  Puerto deportivo de Pasaia             66772   
1        None  Puerto deportivo de Pasaia             66772   
2        None  Puerto deportivo de Pasaia             66772   
3        None  Puerto deportivo de Pasaia             66772   
4        None  Puerto deportivo de Pasaia             66772   

                    geometry  
0  POINT (-1.94818 43.29972)  
1  POINT (-1.94848 43.29912)  
2  POINT (-1.94837 43.29875)  
3  POINT (-1.92805 43.31505)  
4   POINT (-1.92818 43.3139)  


Find shortest path

In [ ]:
# Define a function to compute shortest path length for each row
def compute_shortest_path(row, pt_network):
    # Extract nearest node and nearest POI node
    origin_node = row['nearest_node']
    destination_node = row['nearest_poi_node']
    
    # Calculate the shortest path length between the two nodes
    path_length = pt_network.shortest_path_length(origin_node, destination_node)
    print(f"Shortest path length: {path_length} for row {row.name}")
    
    return path_length

# Apply the function to each row of the GeoDataFrame
nodes_gdf['time_to_nearest_poi'] = nodes_gdf.apply(compute_shortest_path, pt_network=pt_network, axis=1)

Shortest path length: 17.763 for row 0
Shortest path length: 17.763 for row 1
Shortest path length: 17.763 for row 2
Shortest path length: 7.85 for row 3
Shortest path length: 7.85 for row 4
Shortest path length: 9.721 for row 5
Shortest path length: 4.816 for row 6
Shortest path length: 5.684 for row 7
Shortest path length: 5.684 for row 8
Shortest path length: 6.997 for row 9
Shortest path length: 4.057 for row 10
Shortest path length: 24.86 for row 11
Shortest path length: 5.166 for row 12
Shortest path length: 28.534 for row 13
Shortest path length: 28.534 for row 14
Shortest path length: 23.612 for row 15
Shortest path length: 14.879 for row 16
Shortest path length: 28.421 for row 17
Shortest path length: 27.679 for row 18
Shortest path length: 25.8 for row 19
Shortest path length: 7.262 for row 20
Shortest path length: 13.381 for row 21
Shortest path length: 24.567 for row 22
Shortest path length: 14.865 for row 23
Shortest path length: 9.916 for row 24
Shortest path length: 14.9

In [ ]:
# count rows with inf values
print(nodes_gdf[nodes_gdf['time_to_nearest_poi'] == float('inf')].shape[0])

# print the rows with inf values
print(nodes_gdf[nodes_gdf['time_to_nearest_poi'] == float('inf')])

0
Empty GeoDataFrame
Columns: [Referencia, lon, lat, nearest_node, time_to_nearest_poi, block_color, nearest_poi, nearest_poi_node, geometry]
Index: []


In [ ]:
print(nodes_gdf)

      Referencia       lon        lat  nearest_node  time_to_nearest_poi  \
0        8594095 -1.948183  43.299723         29675               17.763   
1        8594099 -1.948484  43.299120         29675               17.763   
2        8594100 -1.948365  43.298748         29675               17.763   
3        8796114 -1.928046  43.315046         64984                7.850   
4        8796136 -1.928175  43.313904         64984                7.850   
...          ...       ...        ...           ...                  ...   
58763    6779028 -2.172469  43.165117         41260               66.209   
58764    6779053 -2.172669  43.165403         20980               66.207   
58765    6779097 -2.172889  43.164341         41264               65.637   
58766    6779001 -2.173457  43.165434         20979               65.430   
58767    6779052 -2.173675  43.165523         20979               65.430   

      block_color                 nearest_poi  nearest_poi_node  \
0            None  P

In [ ]:
# count rows with null values in the time_to_nearest_poi column
print(nodes_gdf['time_to_nearest_poi'].isnull().sum())

0


Assign color depending on time

In [ ]:
# Assign colors to the blocks based on the time it takes to reach the nearest POI
nodes_gdf.loc[nodes_gdf['time_to_nearest_poi'] < 7, 'block_color'] = '#00572a'  # Dark Green
nodes_gdf.loc[(nodes_gdf['time_to_nearest_poi'] >= 7) & (nodes_gdf['time_to_nearest_poi'] < 15), 'block_color'] = '#7CB342'  # Green
nodes_gdf.loc[(nodes_gdf['time_to_nearest_poi'] >= 15) & (nodes_gdf['time_to_nearest_poi'] < 22), 'block_color'] = '#FFFF00'  # Yellow
nodes_gdf.loc[(nodes_gdf['time_to_nearest_poi'] >= 22) & (nodes_gdf['time_to_nearest_poi'] < 30), 'block_color'] = '#FFA500'  # Orange
nodes_gdf.loc[(nodes_gdf['time_to_nearest_poi'] >= 30) & (nodes_gdf['time_to_nearest_poi'] < 40), 'block_color'] = '#D50000'  # Red
nodes_gdf.loc[nodes_gdf['time_to_nearest_poi'] >= 40, 'block_color'] = '#8f0340'  # Burgundy
#nodes_gdf.loc[(nodes_gdf['time_to_nearest_poi'] >= 20) & (nodes_gdf['time_to_nearest_poi'] < 25), 'block_color'] = '#8f0340'  # Dark red
#nodes_gdf.loc[nodes_gdf['time_to_nearest_poi'] >= 25, 'block_color'] = '#6a1717'  # Purple

In [ ]:
# save gdf
nodes_gdf.to_file('./outputs/pt/estetica_nodes_times_pt_newcolors.geojson', driver='GeoJSON')